# 1. SEP28 PREPARATION

In [6]:
!unzip sep28_all_together

unzip:  cannot find or open sep28_all_together, sep28_all_together.zip or sep28_all_together.ZIP.


In [1]:
!ls ./sep28_all_together/

StutterTalk_43_55.wav	     WomenWhoStutter_88_92.wav
StutterTalk_43_56.wav	     WomenWhoStutter_88_93.wav
StutterTalk_43_57.wav	     WomenWhoStutter_88_94.wav
StutterTalk_43_58.wav	     WomenWhoStutter_88_95.wav
StutterTalk_43_59.wav	     WomenWhoStutter_88_96.wav
StutterTalk_4_35.wav	     WomenWhoStutter_88_97.wav
StutterTalk_43_5.wav	     WomenWhoStutter_88_98.wav
StutterTalk_43_60.wav	     WomenWhoStutter_88_99.wav
StutterTalk_43_61.wav	     WomenWhoStutter_88_9.wav
StutterTalk_43_62.wav	     WomenWhoStutter_8_8.wav
StutterTalk_43_63.wav	     WomenWhoStutter_89_0.wav
StutterTalk_43_64.wav	     WomenWhoStutter_89_10.wav
StutterTalk_43_65.wav	     WomenWhoStutter_89_11.wav
StutterTalk_43_66.wav	     WomenWhoStutter_89_12.wav
StutterTalk_43_67.wav	     WomenWhoStutter_89_13.wav
StutterTalk_43_68.wav	     WomenWhoStutter_89_14.wav
StutterTalk_43_69.wav	     WomenWhoStutter_89_15.wav
StutterTalk_4_36.wav	     WomenWhoStutter_89_16.wav
StutterTalk_43_6.wav	     WomenWhoStutter_89_17.wav
S

In [2]:
import os

root_folder = './sep28_all_together/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 21856


In [3]:
import pandas as pd

df_sep = pd.read_csv('SEP_28k_labels.csv')
df_sep.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [3]:
print(df_sep[(df_sep['Show'] == 'WomenWhoStutter') & (df_sep['EpId'] == 99) & (df_sep['ClipId'] == 99)]['NoStutteredWords'])

27324    2
Name: NoStutteredWords, dtype: int64


In [4]:
tmp = df_sep[(df_sep['NoStutteredWords'] == 3) 
                        & (df_sep['Unsure'] == 0)
                        & (df_sep['PoorAudioQuality'] == 0)
                        & (df_sep['Prolongation'] == 0)
                        & (df_sep['Block'] == 0)
                        & (df_sep['SoundRep'] == 0)    
                        & (df_sep['WordRep'] == 0)
                        & (df_sep['DifficultToUnderstand'] == 0)
                        & (df_sep['Interjection'] == 0)
                        & (df_sep['NaturalPause'] == 0)
                        & (df_sep['Music'] == 0)
                        & (df_sep['NoSpeech'] == 0)]

len(df_sep), len(tmp)

(28177, 2479)

In [5]:
col_list_nostutter = [ 'Unsure', 
            'PoorAudioQuality',
            'Prolongation', 
            'Block', 
            'SoundRep',
            'NoStutteredWords',
            'DifficultToUnderstand',
            'Interjection',
            'NaturalPause',
            'Music',
            'NoSpeech']

tmp = df_sep[(df_sep['WordRep'] != 0) & (df_sep[col_list_nostutter].sum(axis=1) == 0)]

len(df_sep), len(tmp)

(28177, 289)

### Creating no stutter files with confidence 3 and count of 258

In [6]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './sep28_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './sep28_balanced_4'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if features[8] == 3 and sum(features) == 3:
        print(show_name,pod_id,clip_id, features,df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)]['NoStutteredWords'])
        count += 1
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
    if count == 258:
        break


print(count)

2023-06-21 17:08:27.751856: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 17:08:27.862354: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-21 17:08:27.862383: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-21 17:08:27.881907: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-21 17:08:30.067758: W tensorflow/stream_executor/platform/de

In [33]:
import os

root_folder = './sep28_balanced_4/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 516


### Creating stutter files for WordRep to be detected with confidence almost 3 and count of 258.

In [4]:
# tmp = df_sep[(df_sep['NoStutteredWords'] == 0) 
#                         & (df_sep['Unsure'] == 0)
#                         & (df_sep['PoorAudioQuality'] == 0)
#                         # & (df_sep['Prolongation'] == 0)
#                         & (df_sep['Block'] == 0)
#                         & ((df_sep['SoundRep'] >= 2) | (df_sep['Interjection'] >= 2) | ((df_sep['Prolongation'] == 0))    
#                         # & (df_sep['WordRep'] >= 2)
#                         & (df_sep['DifficultToUnderstand'] == 0)
#                         # & (df_sep['Interjection'] == 0)
#                         & (df_sep['NaturalPause'] == 0)
#                         & (df_sep['Music'] == 0)
#                         & (df_sep['NoSpeech'] == 0)]

tmp = df_sep[((df_sep['NoStutteredWords'] == 0) 
                        & (df_sep['Unsure'] == 0)
                        & (df_sep['PoorAudioQuality'] == 0)
                        & (df_sep['Prolongation'] == 0)
                        & (df_sep['Block'] == 0)
                        & ((df_sep['SoundRep'] == 0))    
                        & (df_sep['WordRep'] != 0)
                        & (df_sep['DifficultToUnderstand'] == 0)
                        & (df_sep['Interjection'] == 0)
                        & (df_sep['NaturalPause'] == 0)
                        & (df_sep['Music'] == 0)
                        & (df_sep['NoSpeech'] == 0))]

len(df_sep), len(tmp)

(28177, 289)

In [8]:
col_list_stutter = [ 'Unsure', 
            'PoorAudioQuality',
            'Prolongation', 
            'Block', 
            'SoundRep',
            'WordRep',
            'DifficultToUnderstand',
            'Interjection',
            'NaturalPause',
            'Music',
            'NoSpeech']

tmp = df_sep[(df_sep['NoStutteredWords'] == 0) & (df_sep[col_list_stutter].sum(axis=1) == 3)]

len(df_sep), len(tmp)

(28177, 1295)

In [9]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './sep28_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './sep28_balanced_4'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if (features[5] != 0 and sum(features) == features[5]):
        print(show_name,pod_id,clip_id)
        print(features,df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)][['Prolongation','Block','SoundRep','WordRep']])

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)


WomenWhoStutter 42 116
[0 0 0 0 0 3 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
22750             0      0         0        3
WomenWhoStutter 30 94
[0 0 0 0 0 3 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
21717             0      0         0        3
StutterTalk 43 37
[0 0 0 0 0 3 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
12654             0      0         0        3
MyStutteringLife 30 64
[0 0 0 0 0 3 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
6848             0      0         0        3
StutterTalk 25 157
[0 0 0 0 0 3 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
11434             0      0         0        3
WomenWhoStutter 10 161
[0 0 0 0 0 3 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
20317             0      0         0        3
WomenWhoStutter 59 29
[0 0 0 0 0 3 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
24354             0      0         0        3
WomenWhoStutter 6 86
[0 0 

In [5]:
import os

root_folder = './sep28_balanced_4/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)


Total number of files: 516


### Renaming files in sep28_balanced_4 to nostutter (258) and stutter files(258):

In [7]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './sep28_balanced_4/'

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]

    
    if features[8] == 3:
        print(features)
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "nostutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

    elif features[5] != 0:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "stutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

2023-06-21 18:34:55.592943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 18:34:55.723068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-21 18:34:55.723100: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-21 18:34:55.760563: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-21 18:34:57.422116: W tensorflow/stream_executor/platform/de

In [12]:
import os

# Specify the directory where your .wav files are located
directory = './sep28_balanced_4/'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('nostutter'):
        count += 1
    if filename.startswith('stutter'):
        count1 += 1

print(f"Number of nostutter and stutter files: {count} , {count1} and tot {count+count1}")

Number of nostutter and stutter files: 258 , 258 and tot 516


In [9]:
!ls ./sep28_balanced_4/

nostutter_HeStutters_0_22.wav	       stutter_HeStutters_11_125.wav
nostutter_HeStutters_10_8.wav	       stutter_HeStutters_11_128.wav
nostutter_HeStutters_11_67.wav	       stutter_HeStutters_11_45.wav
nostutter_HeStutters_12_133.wav        stutter_HeStutters_12_143.wav
nostutter_HeStutters_12_14.wav	       stutter_HeStutters_12_31.wav
nostutter_HeStutters_12_4.wav	       stutter_HeStutters_12_80.wav
nostutter_HeStutters_1_256.wav	       stutter_HeStutters_15_149.wav
nostutter_HeStutters_14_34.wav	       stutter_HeStutters_15_187.wav
nostutter_HeStutters_14_39.wav	       stutter_HeStutters_19_66.wav
nostutter_HeStutters_17_28.wav	       stutter_HeStutters_1_98.wav
nostutter_HeStutters_17_36.wav	       stutter_HeStutters_21_163.wav
nostutter_HeStutters_17_37.wav	       stutter_HeStutters_21_220.wav
nostutter_HeStutters_18_153.wav        stutter_HeStutters_21_31.wav
nostutter_HeStutters_18_22.wav	       stutter_HeStutters_21_33.wav
nostutter_HeStutters_18_37.wav	       stutter_HeStutters_

### Splitting to train, val, test sets

In [13]:
rounfround(total_filesimport random
import shutil
import os

# Specify the source directory where your files are located
source_directory = './sep28_balanced_4'

# Specify the destination directories for the split files
destination_directory_1 = './train_sep28_4'
destination_directory_2 = './val_sep28_4'
destination_directory_3 = './test_sep28_4'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

count = 0
# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
        count += 1

print(count)


--Return--
None
> /tmp/ipykernel_110/2171575191.py(29)<module>()
     27 split_counts = [int(size * total_files) for size in split_sizes]
     28 import pdb
---> 29 pdb.set_trace()
     30 # Split the files into different lists based on the split counts
     31 file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

516
[412, 51, 51]
2


In [11]:
# Specify the directory where your .wav files are located
directory1 = './train_sep28_4'
directory2 = './val_sep28_4'
directory3 = './test_sep28_4'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    if filename.startswith('nostutter'):
        count_train += 1
    if filename.endswith('.wav'):
        count_train_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    if filename.startswith('nostutter'):
        count_val += 1
    if filename.endswith('.wav'):
        count_val_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    if filename.startswith('nostutter'):
        count_test += 1    
    if filename.endswith('.wav'):
        count_test_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

print('**********Sep28 dataset**********')
print(f"Number of nostutter files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of nostutter files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of nostutter files in the test folder: {count_test} and tot is {count_test_tot}")

**********Sep28 dataset**********
Number of nostutter files in the train folder: 208 and tot is 412
Number of nostutter files in the val folder: 21 and tot is 51
Number of nostutter files in the test folder: 28 and tot is 51


In [ ]:
# import random
# import shutil
# import os
# import tensorflow as tf

# # Specify the source directory where your files are located
# source_directory = './fluency_bank_all_together'

# # Specify the destination directories for the split files
# destination_directory_1 = './fluency_balanced'
# destination_directory_2 = './fluency_inference'

# # Create the destination directories if they don't exist
# for directory in [destination_directory_1, destination_directory_2]:
#     if not os.path.exists(directory):
#         os.makedirs(directory)

# # Shuffle the files in the source directory
# files = os.listdir(source_directory)
# random.shuffle(files)

# # Calculate the number of files for each split
# total_files = len(files)
# split_counts = [800, total_files-800]

# # Split the files into different lists based on the split counts
# file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]]]


# def check_elements(lst, positions):
#     non_zero_count = 0

#     for i in range(len(lst)):
#         if i in positions:
#             if i == 0 and lst[i] == 3:
#                 non_zero_count += 1
#             elif i != 0 and lst[i] >= 0:
#                 non_zero_count += 1
#         elif lst[i] != 0:
#             return False
#     return  non_zero_count >= 1

# specific_positions = [8, 9, 10, 11]

# count = 0
# # Move the files to the destination directories
# for i, file_split in enumerate(file_splits):
#     if i == 0:
#         destination_directory = destination_directory_1
#     else:
#         destination_directory = destination_directory_2

#     for filename in file_split:

#         path_parts = tf.strings.split(filename, '/')
#         path_end = path_parts[-1]
#         file_parts = tf.strings.split(path_end, '_')
#         show_name = file_parts[0].numpy().decode('utf-8')
#         pod_id = int(file_parts[-2])
#         clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
#         features = df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]

#         if check_elements(features, specific_positions):
#             print(show_name,pod_id,clip_id)
#             print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['NoStutteredWords','NaturalPause','Music','NoSpeech']])
                


#             # source_path = os.path.join(source_directory, filename)
#             # destination_path = os.path.join(destination_directory, filename)
#             # shutil.copy2(source_path, destination_path)
    
#             count += 1

# print(count)

# 2. FLUENCY BANK PREPARATION

In [ ]:
!unzip ./fluency_bank_all_together.zip

In [14]:
!ls ./fluency_bank_all_together

FluencyBank_024_111.wav  FluencyBank_096_84.wav   FluencyBank_173_56.wav
FluencyBank_024_112.wav  FluencyBank_096_85.wav   FluencyBank_173_57.wav
FluencyBank_024_113.wav  FluencyBank_096_86.wav   FluencyBank_173_58.wav
FluencyBank_024_114.wav  FluencyBank_096_87.wav   FluencyBank_173_59.wav
FluencyBank_024_115.wav  FluencyBank_096_88.wav   FluencyBank_173_5.wav
FluencyBank_024_116.wav  FluencyBank_096_89.wav   FluencyBank_173_60.wav
FluencyBank_024_117.wav  FluencyBank_096_8.wav	  FluencyBank_173_61.wav
FluencyBank_024_118.wav  FluencyBank_096_90.wav   FluencyBank_173_62.wav
FluencyBank_024_119.wav  FluencyBank_096_91.wav   FluencyBank_173_63.wav
FluencyBank_024_11.wav	 FluencyBank_096_92.wav   FluencyBank_173_64.wav
FluencyBank_024_120.wav  FluencyBank_096_93.wav   FluencyBank_173_65.wav
FluencyBank_024_121.wav  FluencyBank_096_94.wav   FluencyBank_173_66.wav
FluencyBank_024_122.wav  FluencyBank_096_9.wav	  FluencyBank_173_67.wav
FluencyBank_024_123.wav  FluencyBank_097_0.wav	  Fluenc

In [15]:
import os

root_folder = './fluency_bank_all_together/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 4144


In [16]:
import pandas as pd

df_flu = pd.read_csv('fluencybank_labels.csv')
df_flu.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,FluencyBank,10,0,88960,136960,0,0,0,1,0,0,0,0,2,0,0,0
1,FluencyBank,10,1,1271520,1319520,0,0,0,0,0,0,0,0,3,0,0,0
2,FluencyBank,10,2,1813760,1861760,0,0,1,0,0,0,0,0,2,0,0,0
3,FluencyBank,10,3,1842720,1890720,0,0,1,0,0,0,0,0,2,1,0,0
4,FluencyBank,10,4,1893280,1941280,0,0,0,0,0,3,0,0,0,0,0,0


In [4]:
column_location = df_flu.columns.get_loc('NoStutteredWords')
loc_NoStutteredWords = df_flu.columns.get_loc('NoStutteredWords')
loc_NaturalPause = df_flu.columns.get_loc('NaturalPause')
loc_Music = df_flu.columns.get_loc('Music')
loc_NoSpeech = df_flu.columns.get_loc('NoSpeech')

loc_NoStutteredWords,loc_NaturalPause,loc_Music,loc_NoSpeech

(13, 14, 15, 16)

In [5]:
listttt = df_flu[(df_flu['Show'] == 'FluencyBank') & (df_flu['EpId'] == 10) & (df_flu['ClipId'] == 0)].to_numpy()[0]
ffff = [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 0]

loc_NoStutteredWords = df_flu.columns.get_loc('NoStutteredWords')
loc_NaturalPause = df_flu.columns.get_loc('NaturalPause')
loc_Music = df_flu.columns.get_loc('Music')
loc_NoSpeech = df_flu.columns.get_loc('NoSpeech')

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if i == 0 and lst[i] == 3:
                non_zero_count += 1
            elif i != 0 and lst[i] >= 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [8, 9, 10, 11]

check_elements(ffff, specific_positions), listttt[5:]

(True, array([0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0], dtype=object))

### Creating no stutter files with confidence 3 and count of 38 to be equivalent with stutter files 

In [22]:
tmp = df_flu[(df_flu['NoStutteredWords'] == 3) 
                        & (df_flu['Unsure'] == 0)
                        & (df_flu['PoorAudioQuality'] == 0)
                        & (df_flu['Prolongation'] == 0)
                        & (df_flu['Block'] == 0)
                        & (df_flu['SoundRep'] == 0)    
                        & (df_flu['WordRep'] == 0)
                        & (df_flu['DifficultToUnderstand'] == 0)
                        & (df_flu['Interjection'] == 0)
                        & (df_flu['NaturalPause'] == 0)
                        & (df_flu['Music'] == 0)
                        & (df_flu['NoSpeech'] == 0)]

len(df_flu), len(tmp)

(4144, 643)

In [25]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './fluency_bank_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './fluency_balanced_4'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features =  df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if features[8] == 3 and sum(features) == 3:
        print(show_name,pod_id,clip_id)
        print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['NoStutteredWords','NaturalPause','Music','NoSpeech']])
        count += 1

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    if count == 38:
        break

print(count)

FluencyBank 19 38
[0 0 0 0 0 0 0 0 3 0 0 0]      NoStutteredWords  NaturalPause  Music  NoSpeech
319                 3             0      0         0
FluencyBank 118 90
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
2722                 3             0      0         0
FluencyBank 107 20
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
2305                 3             0      0         0
FluencyBank 230 26
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
3796                 3             0      0         0
FluencyBank 63 31
[0 0 0 0 0 0 0 0 3 0 0 0]      NoStutteredWords  NaturalPause  Music  NoSpeech
807                 3             0      0         0
FluencyBank 230 91
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
3861                 3             0      0         0
FluencyBank 147 107
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Musi

In [26]:
import os

root_folder = './fluency_balanced_4/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 38


### Creating stutter files for 'WordRep' to be detected as stuttering situation with confidence almost 3 and count of 38.

In [19]:
tmp = df_flu[(df_flu['NoStutteredWords'] == 0) 
                        & (df_flu['Unsure'] == 0)
                        & (df_flu['PoorAudioQuality'] == 0)
                        & (df_flu['Prolongation'] == 0)
                        & (df_flu['Block'] == 0)
                        & (df_flu['SoundRep'] == 0)    
                        & (df_flu['WordRep'] >= 0)
                        & (df_flu['DifficultToUnderstand'] == 0)
                        & (df_flu['Interjection'] == 0)
                        & (df_flu['NaturalPause'] == 0)
                        & (df_flu['Music'] == 0)
                        & (df_flu['NoSpeech'] == 0)]

len(df_flu), len(tmp)

(4144, 38)

In [17]:
loc_Prolongation = df_flu.columns.get_loc('Prolongation')
loc_Block = df_flu.columns.get_loc('Block')
loc_SoundRep = df_flu.columns.get_loc('SoundRep')
loc_WordRep = df_flu.columns.get_loc('WordRep')
loc_Interjection = df_flu.columns.get_loc('Interjection')

loc_Prolongation, loc_Block, loc_SoundRep, loc_WordRep, loc_Interjection

(7, 8, 9, 10, 12)

In [27]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './fluency_bank_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './fluency_balanced_4'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features =  df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if ((features[5] != 0 and sum(features) == features[5])):
        print(show_name,pod_id,clip_id)
        print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['Prolongation', 'Block','SoundRep','WordRep']])
        count += 1

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)


print(count)

FluencyBank 111 98
[0 0 0 0 0 3 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
2565             0      0         0        3
FluencyBank 230 144
[0 0 0 0 0 3 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
3914             0      0         0        3
FluencyBank 985 170
[0 0 0 0 0 3 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
4118             0      0         0        3
FluencyBank 230 115
[0 0 0 0 0 3 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
3885             0      0         0        3
FluencyBank 111 76
[0 0 0 0 0 3 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
2543             0      0         0        3
FluencyBank 96 47
[0 0 0 0 0 3 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
1730             0      0         0        3
FluencyBank 230 174
[0 0 0 0 0 3 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
3944             0      0         0        3
FluencyBank 81 8
[0 0 0 0 0 3 0 0 0 0 0 0]       Pro

In [28]:
import os

root_folder = './fluency_balanced_4/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 76


### Renaming files in fluency_balanced to nostutter (38) and stutter files(38):

In [29]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './fluency_balanced_4/'

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]

    
    if features[8] == 3:
        print(features)
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "nostutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

    elif features[5] != 0:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "stutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]


In [31]:
import os

# Specify the directory where your .wav files are located
directory = './fluency_balanced_4/'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('nostutter'):
        count += 1
    if filename.startswith('stutter'):
        count1 += 1

print(f"Number of nostutter and stutter files: {count} , {count1} and tot {count+count1}")

Number of nostutter and stutter files: 38 , 38 and tot 76


In [32]:
!ls ./fluency_balanced_4

nostutter_FluencyBank_010_31.wav   stutter_FluencyBank_010_4.wav
nostutter_FluencyBank_010_91.wav   stutter_FluencyBank_016_93.wav
nostutter_FluencyBank_017_26.wav   stutter_FluencyBank_016_98.wav
nostutter_FluencyBank_019_38.wav   stutter_FluencyBank_017_34.wav
nostutter_FluencyBank_024_104.wav  stutter_FluencyBank_017_65.wav
nostutter_FluencyBank_024_134.wav  stutter_FluencyBank_019_41.wav
nostutter_FluencyBank_024_3.wav    stutter_FluencyBank_019_57.wav
nostutter_FluencyBank_024_4.wav    stutter_FluencyBank_019_70.wav
nostutter_FluencyBank_031_46.wav   stutter_FluencyBank_024_161.wav
nostutter_FluencyBank_031_7.wav    stutter_FluencyBank_024_84.wav
nostutter_FluencyBank_046_101.wav  stutter_FluencyBank_081_16.wav
nostutter_FluencyBank_046_20.wav   stutter_FluencyBank_081_7.wav
nostutter_FluencyBank_046_85.wav   stutter_FluencyBank_081_8.wav
nostutter_FluencyBank_063_150.wav  stutter_FluencyBank_089_107.wav
nostutter_FluencyBank_063_171.wav  stutter_FluencyBank_089_81.wav
nostutter_F

### Splitting to train, val, test sets

In [5]:
import random
import shutil
import os

# Specify the source directory where your files are located
source_directory = './fluency_balanced_3'

# Specify the destination directories for the split files
destination_directory_1 = './train_fluency_3'
destination_directory_2 = './val_fluency_3'
destination_directory_3 = './test_fluency_3'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

count = 0
# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
        count += 1

print(count)

In [6]:
# Specify the directory where your .wav files are located
directory1 = './train_fluency_3'
directory2 = './val_fluency_3'
directory3 = './test_fluency_3'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    if filename.startswith('nostutter'):
        count_train += 1
    if filename.endswith('.wav'):
        count_train_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    if filename.startswith('nostutter'):
        count_val += 1
    if filename.endswith('.wav'):
        count_val_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    if filename.startswith('nostutter'):
        count_test += 1    
    if filename.endswith('.wav'):
        count_test_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

print('**********Sep28 dataset**********')
print(f"Number of nostutter files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of nostutter files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of nostutter files in the test folder: {count_test} and tot is {count_test_tot}")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c74a7a1-d505-4b2b-a4fa-efea0c0f20da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>